# Ch4 : Load and Explre Time Series Data

* Pandas ilbrary를 사용하여 csv파일의 Time Series dataset을 로드하는 방법
* data-times를 사용하여 로드된 data를 보고 요약 통계를 계산하는 방법
* 요약 통계 계산과 리뷰방법에 대해서

## Daily Female Births Dataset

In [11]:
import pandas as pd

series = pd.read_csv("daily-total-female-births.csv", header=0, index_col=0, parse_dates=True
                  ,squeeze=True)
print(type(series))
series

<class 'pandas.core.series.Series'>


date
1959-01-01    35
1959-01-02    32
1959-01-03    30
1959-01-04    31
1959-01-05    44
              ..
1959-12-27    37
1959-12-28    52
1959-12-29    48
1959-12-30    55
1959-12-31    50
Name: births, Length: 365, dtype: int64

* read_csv()의 param중 filepath_or_buffer='daily-total-female-births.cvs'는 읽고자하는 파일명으로 그 파일을 읽는 역할, url을 통해 파일을 읽어올 수 있다.
* header=0 은 읽고자하는 파일의 head를 어디로 지정하는가, 여기서는 0을 주어 0 행을 header로 지정
* index_col=0 은 첫 번째 열에(0번열) Time Series에 대한 인덱스 정보가 포함되어 있음을 알려주는 역할. index를 0,1,2 로 주지 않음
* parse_dates=True 는 첫번째 열에 데이터를 구분하는 데 필요한 날짜가 포함되
어 있다는 것을 알려주는 역할
* squeeze=True 는 데이터 열이 하나만 있으므로 Dataframe 대신 Series로 data
를 load하는 역할

In [16]:
print(series['1959-01'])
# 이런식으로 인덱스 검색이 가능하다.

date
1959-01-01    35
1959-01-02    32
1959-01-03    30
1959-01-04    31
1959-01-05    44
1959-01-06    29
1959-01-07    45
1959-01-08    43
1959-01-09    38
1959-01-10    27
1959-01-11    38
1959-01-12    33
1959-01-13    55
1959-01-14    47
1959-01-15    45
1959-01-16    37
1959-01-17    50
1959-01-18    43
1959-01-19    41
1959-01-20    52
1959-01-21    34
1959-01-22    53
1959-01-23    39
1959-01-24    32
1959-01-25    37
1959-01-26    43
1959-01-27    39
1959-01-28    35
1959-01-29    44
1959-01-30    38
1959-01-31    24
Name: births, dtype: int64


In [18]:
series.describe()

count    365.000000
mean      41.980822
std        7.348257
min       23.000000
25%       37.000000
50%       42.000000
75%       46.000000
max       73.000000
Name: births, dtype: float64

# Ch5 : Basic Feature Engineering

* feature engineering time series data의 이론적 근거와 목표
* basic date-time 기반으로 input feature을 개발하는 방법
* 더 정교한 지연(lag) 및 sliding window 요약 통계 기능을 개발하는 방법에 대해
배운다.

## Feature Engineering for Time Series

* input값은 이전의 데이터
* 1. Date Time Features : 각 관찰에 대한 시간 단계 자체의 구성 요소
* 2. Lag Features : 이전 시간 단계의 값
* 3. Window Features : 이전 시간 단계의 window에 대한 값

## Goal of Feature Engineering

* Feature Engineering의 목표 : Supervised Learning Algorithm이 모델링 할 수
있도록 새로운 input feature와 output 간의 이상적인 관계를 제공하는 것 (함수
를 만드는 것)
* Time Series의 경우 input,output의 개념이 없다. 우리는 이것을 변환하여
Supervised Learning을 해야한다.
* 우리는 input, output간의 관계를 모르기에 머신러닝이 필요하고 만약 이를 안
다면 필요 없을 것이다.
* 명확성을 위해 일변량 Time Series dataset을 사용하겠지만 이는 다변량 Time
Series dataset에도 적용할 수 있다.


## Mininum Daily Temperatures Dataset


In [33]:
series_tem = pd.read_csv("daily-minimum-temperatures.csv", header=0, index_col=0,
                        parse_dates= True, squeeze= True)
series_tem

Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
              ... 
1990-12-27    14.0
1990-12-28    13.6
1990-12-29    13.5
1990-12-30    15.7
1990-12-31    13.0
Name: Temp, Length: 3650, dtype: float64

In [39]:
from pandas import DataFrame
dataframe = DataFrame()
dataframe['month'] = [series_tem.index[i].month for i in range(len(series_tem))]
dataframe['day'] = [series_tem.index[i].day for i in range(len(series_tem))]
dataframe['temperature'] = [series_tem[i] for i in range(len(series_tem))]
dataframe

,month,day,temperature
0,1,1,20.7
1,1,2,17.9
2,1,3,18.8
3,1,4,14.6
4,1,5,15.8
...,...,...,...
3645,12,27,14.0
3646,12,28,13.6
3647,12,29,13.5
3648,12,30,15.7


## Lag Feature
* Lag Feature은 Time Series problem을 Supervised learning problem으로 변환
하는 고전적인 방법
* 현재 시간 (t)가 주어지면 다음 시간 (t+1)에 값을 예측하는 것
* shift() 함수를 사용하면 dataset이 한 칸씩 이동 되며, t행에 nan값이 추가된다.

In [64]:
from pandas import concat

temps = DataFrame(series_tem.values)
shifted = temps.shift(1)
dataframe = concat([shifted, temps], axis=1)
dataframe
dataframe.columns = ['t', 't+1']
dataframe.head()

,t,t+1
0,NaN,20.7
1,20.7,17.9
2,17.9,18.8
3,18.8,14.6
4,14.6,15.8


In [90]:
dataframe = concat([temps.shift(3), temps.shift(2), temps.shift(1), temps], axis=1)
dataframe.columns = ['t-2', 't-1', 't', 't+1']
print(dataframe)

       t-2   t-1     t   t+1
0      NaN   NaN   NaN  20.7
1      NaN   NaN  20.7  17.9
2      NaN  20.7  17.9  18.8
3     20.7  17.9  18.8  14.6
4     17.9  18.8  14.6  15.8
...    ...   ...   ...   ...
3645  10.0  12.9  14.6  14.0
3646  12.9  14.6  14.0  13.6
3647  14.6  14.0  13.6  13.5
3648  14.0  13.6  13.5  15.7
3649  13.6  13.5  15.7  13.0

[3650 rows x 4 columns]


* 위 data는 windowsize 가 3이다. windowsize를 바꿔가며 어떤 모델이 좋은 것인지 확인해야된다.

## +Sliding window algorithm

In [85]:
def max_sub_array(arr, k):
    window_sum = 0
    max_sum = 0
    window_start = 0
    
    for window_end in range(len(arr)):
        window_sum += arr[window_end]
        
        if window_end >= (k-1):
            max_sum = max(max_sum, window_sum)
            window_sum -= arr[window_start]
            window_start +=1
            
    return max_sum

max_sub_array([2, 4, 7, 10, 8, 4], 3)

# 3개의 윈도우 배열이 우측으로 한칸씩 움직일때, 3개를 전부 다시 사용하지 않고 
# 첫번째 한칸 뺴고 두칸은 재활용, 새로운칸 추가
# 기존 window 방식은 복잡도가 O(NW) 인데 요 방식은 O(N)

25

## Rolling Window Statistics

* sliding window의 값에 대한 요약 통계를 계산하고 dataset의 feature로 포함할
수 있다. 이것은 Rolling mean이라는 것으로 유용하게 수행할 수 있다.

* DataFrame에 rolling()이라는 내장함수가 있고 window 값에 따라 그 크기의
mean을 저장

In [105]:
series = pd.read_csv('daily-minimum-temperatures.csv', header=0, index_col=0,
                 parse_dates=True, squeeze=True)
temps = DataFrame(series.values)
shifted = temps.shift(1)
shifted
means = shifted.rolling(window=2).mean()
dataframe = concat([means, shifted, temps], axis=1)
dataframe

# rolling(window) 함수는 원래 1차원배열 데이터에서 1~window 까지 rollig을 window index에 놓는듯

,0,0,0
0,NaN,NaN,20.7
1,NaN,20.7,17.9
2,19.30,17.9,18.8
3,18.35,18.8,14.6
4,16.70,14.6,15.8
...,...,...,...
3645,13.75,14.6,14.0
3646,14.30,14.0,13.6
3647,13.80,13.6,13.5
3648,13.55,13.5,15.7


In [113]:
min_max = temps.rolling(window=3)
dataframe = concat([min_max.min(), min_max.mean(), min_max.max(), temps], axis=1)
dataframe

,0,0,0,0
0,NaN,NaN,NaN,20.7
1,NaN,NaN,NaN,17.9
2,17.9,19.133333,20.7,18.8
3,14.6,17.100000,18.8,14.6
4,14.6,16.400000,18.8,15.8
...,...,...,...,...
3645,12.9,13.833333,14.6,14.0
3646,13.6,14.066667,14.6,13.6
3647,13.5,13.700000,14.0,13.5
3648,13.5,14.266667,15.7,15.7


In [109]:
m

,0
0,NaN
1,NaN
2,17.9
3,14.6
4,14.6
...,...
3645,12.9
3646,13.6
3647,13.5
3648,13.5
